##### Copyright 2023 The IREE Authors

In [1]:
#@title Licensed under the Apache License v2.0 with LLVM Exceptions.
# See https://llvm.org/LICENSE.txt for license information.
# SPDX-License-Identifier: Apache-2.0 WITH LLVM-exception

# <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/10/PyTorch_logo_icon.svg/640px-PyTorch_logo_icon.svg.png" height="20px"> PyTorch Just-in-time (JIT) workflows using <img src="https://raw.githubusercontent.com/iree-org/iree/main/docs/website/docs/assets/images/IREE_Logo_Icon_Color.svg" height="20px"> IREE

This notebook shows how to use [iree-turbine](https://github.com/iree-org/iree-turbine) for eager execution within a PyTorch session using [IREE](https://github.com/iree-org/iree) and [torch-mlir](https://github.com/llvm/torch-mlir) under the covers.

## Setup

In [2]:
%%capture
#@title Uninstall existing packages
#   This avoids some warnings when installing specific PyTorch packages below.
!python -m pip uninstall -y fastai torchaudio torchdata torchtext torchvision

In [3]:
#@title Install Pytorch 2.8+ (supports Python 3.12)
!python -m pip install torch --index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://download.pytorch.org/whl/cpu


In [4]:
#@title Install iree-turbine
!python -m pip install iree-turbine

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.7/306.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 8.2 MB/s eta 0:00:00


In [5]:
#@title Report version information
!echo "Installed iree-turbine, $(python -m pip show iree_turbine | grep Version)"

!echo -e "\nInstalled IREE, compiler version information:"
!iree-compile --version

import torch
print("\nInstalled PyTorch, version:", torch.__version__)

Installed iree-turbine, Version: 3.7.0

Installed IREE, compiler version information:
IREE (https://iree.dev):
  IREE compiler version 3.7.1 @ cb2048e0e7694c76ac1af36d5e3450aa3625995c
  LLVM version 22.0.0git
  Optimized build

Installed PyTorch, version: 2.8.0+cu126


## Sample JIT workflow

1. **(Optional)** Set `TURBINE_LOG_LEVEL=debug` to see verbose compilation output including intermediate MLIR IR
2. Define a program using `torch.nn.Module`
3. Run `torch.compile(module, backend="turbine_cpu")`
4. Use the resulting `OptimizedModule` as you would a regular `nn.Module`

Useful documentation:

* [PyTorch Modules](https://pytorch.org/docs/stable/notes/modules.html) (`nn.Module`) as building blocks for stateful computation
* [`torch.compile`](https://pytorch.org/docs/stable/generated/torch.compile.html) as an interface to TorchDynamo and optimizing using backend compilers like Turbine

In [6]:
#@title Enable debug logging to see MLIR output
%env TURBINE_LOG_LEVEL=debug

env: TURBINE_LOG_LEVEL=debug


In [7]:
torch.manual_seed(0)

class LinearModule(torch.nn.Module):
  def __init__(self, in_features, out_features):
    super().__init__()
    self.weight = torch.nn.Parameter(torch.randn(in_features, out_features))
    self.bias = torch.nn.Parameter(torch.randn(out_features))

  def forward(self, input):
    return (input @ self.weight) + self.bias

linear_module = LinearModule(4, 3)

In [8]:
opt_linear_module = torch.compile(linear_module, backend="turbine_cpu")
print("Compiled module using Turbine. New module type is", type(opt_linear_module))

Compiled module using Turbine. New module type is <class 'torch._dynamo.eval_frame.OptimizedModule'>


In [9]:
args = torch.randn(4)
turbine_output = opt_linear_module(args)

print("Weight:", linear_module.weight)
print("Bias:", linear_module.bias)
print("Args:", args)
print("Output:", turbine_output)

DEBUG 10-05 09:26:00 [base.py:24] Traced Graph Module:
<lambda>()



def forward(self, arg0_1, arg1_1, arg2_1):
    unsqueeze = torch.ops.aten.unsqueeze.default(arg1_1, 0);  arg1_1 = None
    mm = torch.ops.aten.mm.default(unsqueeze, arg0_1);  arg0_1 = None
    squeeze = torch.ops.aten.squeeze.dim(mm, 0);  mm = None
    add = torch.ops.aten.add.Tensor(squeeze, arg2_1);  squeeze = arg2_1 = None
    return (add, unsqueeze)
    
# To see more debug info, please use `graph_module.print_readable()`
DEBUG 10-05 09:26:00 [base.py:28] Successfully imported gm to mlir:
module {
  func.func @main(%arg0: !torch.vtensor<[4,3],f32>, %arg1: !torch.vtensor<[4],f32>, %arg2: !torch.vtensor<[3],f32>) -> (!torch.vtensor<[3],f32>, !torch.vtensor<[1,4],f32>) {
    %int0 = torch.constant.int 0
    %0 = torch.aten.unsqueeze %arg1, %int0 : !torch.vtensor<[4],f32>, !torch.int -> !torch.vtensor<[1,4],f32>
    %1 = torch.aten.mm %0, %arg0 : !torch.vtensor<[1,4],f32>, !torch.vtensor<[4,3],f32> -> !torch.vtensor<[

Weight: Parameter containing:
tensor([[ 1.5410, -0.2934, -2.1788],
        [ 0.5684, -1.0845, -1.3986],
        [ 0.4033,  0.8380, -0.7193],
        [-0.4033, -0.5966,  0.1820]], requires_grad=True)
Bias: Parameter containing:
tensor([-0.8567,  1.1006, -1.0712], requires_grad=True)
Args: tensor([ 0.1227, -0.5663,  0.3731, -0.8920])
Output: tensor([-0.4792,  2.5237, -0.9772], grad_fn=<CompiledFunctionBackward>)
